In [1]:
import sys
import torch
import numpy as np
import pandas as ps
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn import metrics

import albumentations as alb
import albumentations.pytorch


sys.path.append("..")
device = torch.device("cuda:0")

In [2]:
from src.models import BinaryEfficientNet
from src.datasets import ImagesDataset

ttas = [
    # original
    alb.Compose([
        alb.Resize(512, 512),
        alb.Normalize(),
        alb.pytorch.ToTensorV2(),
    ]),
    # horizontal flipped
    alb.Compose([
        alb.Resize(512, 512),
        alb.HorizontalFlip(p=1),
        alb.Normalize(),
        alb.pytorch.ToTensorV2(),
    ]),
    # vertical flipped
    alb.Compose([
        alb.Resize(512, 512),
        alb.VerticalFlip(p=1),
        alb.Normalize(),
        alb.pytorch.ToTensorV2(),
    ]),
]

len(ttas)

/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/nbformat/notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size

3

In [3]:
model = BinaryEfficientNet("efficientnet-b0")
model.load_state_dict(
    torch.load("../logs/efficientnet-b0-magic-constant/checkpoints/best.pth", map_location="cpu")["model_state_dict"]
)
model = model.to(device)
model = model.eval()

Loaded pretrained weights for efficientnet-b0


In [4]:
import glob

In [5]:
test_images = glob.glob("../data/resized_data/Test/*.jpg")

len(test_images), test_images[:2]

(5000,
 ['../data/resized_data/Test/1787.jpg', '../data/resized_data/Test/0312.jpg'])

In [6]:
tta_preds = []
for transform_idx, transform in enumerate(ttas, 1):

    dataset = ImagesDataset(test_images, transforms=transform)
    loader = DataLoader(dataset, batch_size=64, num_workers=16)

    preds = []
    with torch.no_grad():
        for batch in tqdm(loader, desc=f"TTA index - {transform_idx}"):
            batch = batch.to(device)
            out = torch.sigmoid(model(batch)).detach().cpu().numpy().flatten()
            preds.append(out)

    preds = np.concatenate(preds)
    tta_preds.append(preds)

TTA index: 3: 100%|██████████| 79/79 [00:36<00:00,  2.15it/s]


In [14]:
avg_pred = np.mean(np.stack(tta_preds), 0)
avg_pred.shape, avg_pred

((5000,),
 array([0.5759569 , 0.76441056, 0.6511685 , ..., 0.80173016, 0.573744  ,
        0.66806084], dtype=float32))

In [23]:
submission = ps.DataFrame.from_dict({
    "Id": [file.rsplit("/", 1)[1] for file in test_images],
    "Label": avg_pred
}).sort_values(by="Id").reset_index(drop=True)

print(submission.shape)
submission.head()

(5000, 2)


,Id,Label
0,0001.jpg,0.694623
1,0002.jpg,0.648925
2,0003.jpg,0.724082
3,0004.jpg,0.686711
4,0005.jpg,0.648012


In [24]:
submission.to_csv("../submissions/bin_en_b0_hvflip.csv", index=False)

In [25]:
!head ../submissions/bin_en_b0_hvflip.csv

Id,Label
0001.jpg,0.6946228
0002.jpg,0.64892507
0003.jpg,0.7240815
0004.jpg,0.6867114
0005.jpg,0.6480124
0006.jpg,0.75760317
0007.jpg,0.8171317
0008.jpg,0.6424361
0009.jpg,0.5988344
